In [ ]:
import pandas as pd
import requests
import os
import re
import time

# Configuration
# df = pd.read_csv('Spring_2025_ENGR-2010-002.csv')

csv_file = 'YourExcelFile.csv'
df = pd.read_csv(csv_file)


# Filter the dataset to only include rows with Mime type "application/PDF" and Ocred:2 equal to 0
df = df[(df['Mime type'] == 'application/pdf') & (df['Ocred:2'] == 0)]
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 12 entries, 8 to 39
Data columns (total 46 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Id                       12 non-null     object 
 1   Name                     12 non-null     object 
 2   Mime type                12 non-null     object 
 3   Score                    12 non-null     float64
 4   Deleted at               0 non-null      float64
 5   Library reference        0 non-null      float64
 6   Url                      12 non-null     object 
 7   Checked on               12 non-null     object 
 8   AlternativeText:2        12 non-null     int64  
 9   Contrast:2               12 non-null     int64  
 10  HeadingsHigherLevel:3    12 non-null     int64  
 11  HeadingsPresence:2       12 non-null     int64  
 12  HeadingsSequential:3     12 non-null     int64  
 13  HeadingsStartAtOne:3     12 non-null     int64  
 14  HtmlBrokenLink:2         12 non-n

In [ ]:
file_id_column = "Id"
filename_column = "Name"

# Canvas API token and base URL
canvas_token = "1009~2uRhmABNCBRuTBwfGfWmhP6JZxARv96PTBYyutQMvRD3cvf3tV6B4WkVCXLYGBXH"
base_url = "https://usu.instructure.com"

headers = {
    "Authorization": f"Bearer {canvas_token}"
}

# Folder for downloads
output_folder = "Downloads"
os.makedirs(output_folder, exist_ok=True)

# Function to sanitize filenames (removes characters not allowed on Windows)
def sanitize_filename(name):
    return re.sub(r'[\\/*?:"<>|]', "", name)

# Loop through each row in the filtered DataFrame
for index, row in df.iterrows():
    file_id = row[file_id_column]
    file_name = sanitize_filename(str(row[filename_column]))

    if pd.isna(file_id):
        print(f"Row {index} has no File ID. Skipping.")
        continue

    # Step 1: Get file metadata from Canvas API
    file_api_url = f"{base_url}/api/v1/files/{int(file_id)}"
    meta_resp = requests.get(file_api_url, headers=headers)

    if meta_resp.status_code != 200:
        print(f"Failed to retrieve metadata for file ID {file_id} (Status: {meta_resp.status_code}). Skipping.")
        continue

    file_info = meta_resp.json()
    download_url = file_info.get("url")
    expected_size = file_info.get("size")

    if not download_url:
        print(f"No download URL found for file ID {file_id}. Skipping.")
        continue

    # Step 2: Download the file using streaming
    print(f"Downloading {file_name} from {download_url}")
    download_resp = requests.get(download_url, headers=headers, stream=True)

    if download_resp.status_code != 200:
        print(f"Failed to download {file_name} (Status: {download_resp.status_code}).")
        continue

    # Check Content-Type for extra confirmation
    content_type = download_resp.headers.get("Content-Type", "")
    if "application/pdf" not in content_type:
        print(f"Warning: {file_name} returned unexpected Content-Type: {content_type}")

    # Save the file in chunks
    filepath = os.path.join(output_folder, file_name)
    with open(filepath, "wb") as f:
        for chunk in download_resp.iter_content(chunk_size=8192):
            if chunk:
                f.write(chunk)
    download_resp.close()

    # Verify file size after download
    actual_size = os.path.getsize(filepath)
    if expected_size and actual_size < expected_size:
        print(f"Downloaded {file_name} is smaller than expected (Expected: {expected_size} bytes, Got: {actual_size} bytes).")
    else:
        print(f"Downloaded: {file_name} ({actual_size} bytes) successfully.")

    # Optional: short delay to avoid rate limiting
    time.sleep(1)


Downloaded: 3710 score distribution sheets updated 19 Feb 2020-1.pdf (1535819 bytes) successfully.
Downloaded: 3710 score distribution sheets updated 19 Feb 2020.pdf (1535819 bytes) successfully.
Downloaded: 3710 score distribution sheets updated 26 January 2022-1-1.pdf (5191066 bytes) successfully.
Downloaded: 3710 score distribution sheets updated 26 January 2022-1.pdf (5191066 bytes) successfully.
Downloaded: 3710 score distribution sheets updated 26 January 2022-2.pdf (5191066 bytes) successfully.
Downloaded: 3710 score distribution sheets updated 26 January 2022.pdf (5191066 bytes) successfully.
Downloaded: CHEM 3710 Lab Report Score Distribution Sheets-1-1.pdf (1602908 bytes) successfully.
Downloaded: CHEM 3710 Lab Report Score Distribution Sheets-1.pdf (1602908 bytes) successfully.
Downloaded: CHEM 3710 Lab Report Score Distribution Sheets-2.pdf (1513248 bytes) successfully.
Downloaded: CHEM 3710 Lab Report Score Distribution Sheets.pdf (1513248 bytes) successfully.
Downloaded: 

In [ ]:
import zipfile
from google.colab import files

def zip_and_download_folder(folder_path, zip_filename):
  """Zips a folder and downloads the zip file."""

  with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, _, filenames in os.walk(folder_path):
      for filename in filenames:
        filepath = os.path.join(root, filename)
        zipf.write(filepath, os.path.relpath(filepath, folder_path))

  files.download(zip_filename)


folder_to_zip = 'Downloads'
zip_filename = 'downloads.zip'

zip_and_download_folder(folder_to_zip, zip_filename)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>